![header](images/header.jpg)

# Predicting Condition of Tanzanian Water Wells

**Author:** [Jonathan Fetterolf](mailto:jonathan.fetterolf@gmail.com)

## Overview

## Business Understanding and Business Problem
Using data from each waterpoint, can you predict which pumps are functional, which need some repairs, and which don't work at all? Data driven predictions will lead to a better maintenance operations and will ensure clean and potable water is available to communities across Tanzania. 

## Data Understanding
Data for this project is from [Taarifa](http://taarifa.org/) and the [Tanzanian Ministry of Water](http://maji.go.tz/).

## Exploratory Data Analyis

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

from sklearn.dummy import DummyClassifier

## Import Initial Data

In [2]:
df = pd.read_csv('data/training_set_values.csv', index_col='id')

## Functions


Creating a function and transformer that returns numeric columns. 

In [3]:
def grab_numeric(df):
    return df.select_dtypes(include=['float', 'int'])

In [4]:
GrabNumeric = FunctionTransformer(grab_numeric)

Numeric Pipeline

In [5]:
numeric_pipe = Pipeline([
    ('numeric_impute', SimpleImputer(strategy='mean')),
    ('ss', StandardScaler())
])

In [6]:
def null_info(col_name, df_name=df):
    '''
    give this function a column name and name of the DataFrame to
    receive a printout with the name of the column and percentage
    of rows with null values in the DataFrame
    '''
    percent = round((df_name[col_name].isna().sum() / df_length) * 100, ndigits=2)
    values = df_name[col_name].value_counts().head()
    length = len(list(df_name[col_name].unique()))
    col_type = df_name[col_name].dtypes
    return print(f'''This column consists of {col_type}s and null values.\n
The percentage of missing rows in the {col_name} \
column is {percent}%.\n
The top values in {col_name} are:\n
{values}\n
There are {length} unique values in this column.
    ''')

## Data

In [7]:
df_length = len(df)
df_length

59400

In [8]:
col_names = list(df.columns)
num_cols = len(col_names)

print(f'The column names are: {col_names}\n Number of columns: {num_cols}')

The column names are: ['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']
 Number of columns: 39


### About the columns:
amount_tsh - Total static head (amount water available to waterpoint)<br>
date_recorded - The date the row was entered<br>
funder - Who funded the well<br>
gps_height - Altitude of the well<br>
installer - Organization that installed the well<br>
longitude - GPS coordinate<br>
latitude - GPS coordinate<br>
wpt_name - Name of the waterpoint if there is one<br>
num_private - <br>
basin - Geographic water basin<br>
subvillage - Geographic location<br>
region - Geographic location<br>
region_code - Geographic location (coded)<br>
district_code - Geographic location (coded)<br>
lga - Geographic location<br>
ward - Geographic location<br>
population - Population around the well<br>
public_meeting - True/False<br>
recorded_by - Group entering this row of data<br>
scheme_management - Who operates the waterpoint<br>
scheme_name - Who operates the waterpoint<br>
permit - If the waterpoint is permitted<br>
construction_year - Year the waterpoint was constructed<br>
extraction_type - The kind of extraction the waterpoint uses<br>
extraction_type_group - The kind of extraction the waterpoint uses<br>
extraction_type_class - The kind of extraction the waterpoint uses<br>
management - How the waterpoint is managed<br>
management_group - How the waterpoint is managed<br>
payment - What the water costs<br>
payment_type - What the water costs<br>
water_quality - The quality of the water<br>
quality_group - The quality of the water<br>
quantity - The quantity of water<br>
quantity_group - The quantity of water<br>
source - The source of the water<br>
source_type - The source of the water<br>
source_class - The source of the water<br>
waterpoint_type - The kind of waterpoint<br>
waterpoint_type_group - The kind of waterpoint<br>

In [9]:
# let's see what we're working with
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


The condition labels are in a separate DataFrame so we need to bring them in.

In [10]:
df_target = pd.read_csv('data/training_set_labels.csv', index_col='id')

In [11]:
df_target.columns, df_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 928.1+ KB


(Index(['status_group'], dtype='object'), None)

In [12]:
# looks like we have the same number of rows.
df_target.shape, df.shape

((59400, 1), (59400, 39))

In [13]:
df = df.join(df_target)

In [14]:
# okay, our data is consolidated
df.shape

(59400, 40)

What are we looking at?

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

In [16]:
df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


### Addressing Null Values

In [17]:
df.isna().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

#### 'funder'

In [18]:
null_info('funder')

This column consists of objects and null values.

The percentage of missing rows in the funder column is 6.12%.

The top values in funder are:

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Name: funder, dtype: int64

There are 1898 unique values in this column.
    


#### 'installer'

In [19]:
null_info('installer')

This column consists of objects and null values.

The percentage of missing rows in the installer column is 6.15%.

The top values in installer are:

DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
Name: installer, dtype: int64

There are 2146 unique values in this column.
    


#### 'subvillage'

In [20]:
null_info('subvillage')

This column consists of objects and null values.

The percentage of missing rows in the subvillage column is 0.62%.

The top values in subvillage are:

Madukani    508
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
Name: subvillage, dtype: int64

There are 19288 unique values in this column.
    


#### 'public_meeting'

In [21]:
null_info('public_meeting')

This column consists of objects and null values.

The percentage of missing rows in the public_meeting column is 5.61%.

The top values in public_meeting are:

True     51011
False     5055
Name: public_meeting, dtype: int64

There are 3 unique values in this column.
    


#### 'scheme_management'

In [22]:
null_info('scheme_management')

This column consists of objects and null values.

The percentage of missing rows in the scheme_management column is 6.53%.

The top values in scheme_management are:

VWC                36793
WUG                 5206
Water authority     3153
WUA                 2883
Water Board         2748
Name: scheme_management, dtype: int64

There are 13 unique values in this column.
    


#### 'scheme_name'

In [23]:
null_info('scheme_name')

This column consists of objects and null values.

The percentage of missing rows in the scheme_name column is 47.42%.

The top values in scheme_name are:

K                682
None             644
Borehole         546
Chalinze wate    405
M                400
Name: scheme_name, dtype: int64

There are 2697 unique values in this column.
    


I will drop this column because over 47% of the rows have missing data and the 'scheme_managment' column has the same description of, "Who operates the waterpoint."

#### 'permit'

In [24]:
null_info('permit')

This column consists of objects and null values.

The percentage of missing rows in the permit column is 5.14%.

The top values in permit are:

True     38852
False    17492
Name: permit, dtype: int64

There are 3 unique values in this column.
    


## Baseline Model

## First Simple Model